In [1]:
!pip install scanpy
!pip install KDEpy
!pip install leidenalg

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import BacSC.util_probe as up
import BacSC.util as ut
import BacSC.NB_est as nb
import BacSC.countsplit as cs
import BacSC.scDEED as scd
import BacSC.clustering_opt as co

import warnings
warnings.filterwarnings("ignore")

In [3]:
import importlib

In [4]:
data_path = "../../data/MOBA_scRNAseq/outs_S3"
data_gene = sc.read_h5ad(data_path + "/filtered_data_maxpool_processed.h5ad")
data_gene

AnnData object with n_obs × n_vars = 1544 × 5553
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'outlier', 'n_genes', 'total_counts_norm', 'total_counts_scale', 'leiden_opt'
    var: 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'var_counts', 'is_scd_outlier', 'nb_overdisp', 'nb_overdisp_cutoff', 'nb_mean', 'nb_umi', 'Intercept_step1_sct', 'log_umi_step1_sct', 'dispersion_step1_sct', 'mean', 'std', 'Locus tag', 'Protein accession', 'Name', 'Symbol'
    uns: 'BacSC_params', 'leiden_opt', 'leiden_opt_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'vst_counts'
    obsp: 'connectivities', 'distances'

In [9]:
k_opt = data_gene.uns["BacSC_params"]["k_opt"]
n_neighbors_desired = data_gene.uns["BacSC_params"]["n_neighbors_opt"]
min_dist_desired = data_gene.uns["BacSC_params"]["min_dist_opt"]

print(n_neighbors_desired)
print(min_dist_desired)

50
0.3


In [ ]:
importlib.reload(scd)
n_neighbors_opt = 0
min_dist_opt = 0

n_neighborss = [10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 150, 200, 250]
min_dists = [0.05, 0.1, 0.3, 0.5, 0.7]

while (n_neighbors_opt != n_neighbors_desired) or (min_dist_opt != min_dist_desired):
    rng = np.random.default_rng()
    seed = rng.choice(1001)
    print(seed)

    scdeed_result, rel_scores, null_rel_scores, reliabilities, n_neighbors_opt, min_dist_opt = scd.scdeed_parameter_selection(data_gene, n_neighborss, min_dists, n_pcs=k_opt, rng_seed=seed, embedding_method="UMAP", similarity_percent=0.5, save_path="S3_scdeed")
    
    print(f"Optimal number of neighbors: {n_neighbors_opt}")
    print(f"Optimal min_dist: {min_dist_opt}")

868
calculating (10, 0.05) - run 1/85
calculating (10, 0.1) - run 2/85
calculating (10, 0.3) - run 3/85
calculating (10, 0.5) - run 4/85
calculating (10, 0.7) - run 5/85
calculating (15, 0.05) - run 6/85
calculating (15, 0.1) - run 7/85
calculating (15, 0.3) - run 8/85
calculating (15, 0.5) - run 9/85
calculating (15, 0.7) - run 10/85
calculating (20, 0.05) - run 11/85
calculating (20, 0.1) - run 12/85
calculating (20, 0.3) - run 13/85
calculating (20, 0.5) - run 14/85
calculating (20, 0.7) - run 15/85
calculating (25, 0.05) - run 16/85
calculating (25, 0.1) - run 17/85
calculating (25, 0.3) - run 18/85
calculating (25, 0.5) - run 19/85
calculating (25, 0.7) - run 20/85
calculating (30, 0.05) - run 21/85
calculating (30, 0.1) - run 22/85
calculating (30, 0.3) - run 23/85
calculating (30, 0.5) - run 24/85
calculating (30, 0.7) - run 25/85
calculating (35, 0.05) - run 26/85
calculating (35, 0.1) - run 27/85
calculating (35, 0.3) - run 28/85
calculating (35, 0.5) - run 29/85
calculating (

calculating (100, 0.1) - run 67/85
calculating (100, 0.3) - run 68/85
calculating (100, 0.5) - run 69/85
calculating (100, 0.7) - run 70/85
calculating (150, 0.05) - run 71/85
calculating (150, 0.1) - run 72/85
calculating (150, 0.3) - run 73/85
calculating (150, 0.5) - run 74/85
calculating (150, 0.7) - run 75/85
calculating (200, 0.05) - run 76/85
calculating (200, 0.1) - run 77/85
calculating (200, 0.3) - run 78/85
calculating (200, 0.5) - run 79/85
calculating (200, 0.7) - run 80/85
calculating (250, 0.05) - run 81/85
calculating (250, 0.1) - run 82/85
calculating (250, 0.3) - run 83/85
calculating (250, 0.5) - run 84/85
calculating (250, 0.7) - run 85/85
Optimal number of neighbors: 20
Optimal min_dist: 0.3
886
calculating (10, 0.05) - run 1/85
calculating (10, 0.1) - run 2/85
calculating (10, 0.3) - run 3/85
calculating (10, 0.5) - run 4/85
calculating (10, 0.7) - run 5/85
calculating (15, 0.05) - run 6/85
calculating (15, 0.1) - run 7/85
calculating (15, 0.3) - run 8/85
calculat

calculating (60, 0.05) - run 46/85
calculating (60, 0.1) - run 47/85
calculating (60, 0.3) - run 48/85
calculating (60, 0.5) - run 49/85
calculating (60, 0.7) - run 50/85
calculating (70, 0.05) - run 51/85
calculating (70, 0.1) - run 52/85
calculating (70, 0.3) - run 53/85
calculating (70, 0.5) - run 54/85
calculating (70, 0.7) - run 55/85
calculating (80, 0.05) - run 56/85
calculating (80, 0.1) - run 57/85
calculating (80, 0.3) - run 58/85
calculating (80, 0.5) - run 59/85
calculating (80, 0.7) - run 60/85
calculating (90, 0.05) - run 61/85
calculating (90, 0.1) - run 62/85
calculating (90, 0.3) - run 63/85
calculating (90, 0.5) - run 64/85
calculating (90, 0.7) - run 65/85
calculating (100, 0.05) - run 66/85
calculating (100, 0.1) - run 67/85
calculating (100, 0.3) - run 68/85
calculating (100, 0.5) - run 69/85
calculating (100, 0.7) - run 70/85
calculating (150, 0.05) - run 71/85
calculating (150, 0.1) - run 72/85
calculating (150, 0.3) - run 73/85
calculating (150, 0.5) - run 74/85

calculating (30, 0.7) - run 25/85
calculating (35, 0.05) - run 26/85
calculating (35, 0.1) - run 27/85
calculating (35, 0.3) - run 28/85
calculating (35, 0.5) - run 29/85
calculating (35, 0.7) - run 30/85
calculating (40, 0.05) - run 31/85
calculating (40, 0.1) - run 32/85
calculating (40, 0.3) - run 33/85
calculating (40, 0.5) - run 34/85
calculating (40, 0.7) - run 35/85
calculating (45, 0.05) - run 36/85
calculating (45, 0.1) - run 37/85
calculating (45, 0.3) - run 38/85
calculating (45, 0.5) - run 39/85
calculating (45, 0.7) - run 40/85
calculating (50, 0.05) - run 41/85
calculating (50, 0.1) - run 42/85
calculating (50, 0.3) - run 43/85
calculating (50, 0.5) - run 44/85
calculating (50, 0.7) - run 45/85
calculating (60, 0.05) - run 46/85
calculating (60, 0.1) - run 47/85
calculating (60, 0.3) - run 48/85
calculating (60, 0.5) - run 49/85
calculating (60, 0.7) - run 50/85
calculating (70, 0.05) - run 51/85
calculating (70, 0.1) - run 52/85
calculating (70, 0.3) - run 53/85
calculat

calculating (10, 0.5) - run 4/85
calculating (10, 0.7) - run 5/85
calculating (15, 0.05) - run 6/85
calculating (15, 0.1) - run 7/85
calculating (15, 0.3) - run 8/85
calculating (15, 0.5) - run 9/85
calculating (15, 0.7) - run 10/85
calculating (20, 0.05) - run 11/85
calculating (20, 0.1) - run 12/85
calculating (20, 0.3) - run 13/85
calculating (20, 0.5) - run 14/85
calculating (20, 0.7) - run 15/85
calculating (25, 0.05) - run 16/85
calculating (25, 0.1) - run 17/85
calculating (25, 0.3) - run 18/85
calculating (25, 0.5) - run 19/85
calculating (25, 0.7) - run 20/85
calculating (30, 0.05) - run 21/85
calculating (30, 0.1) - run 22/85
calculating (30, 0.3) - run 23/85
calculating (30, 0.5) - run 24/85
calculating (30, 0.7) - run 25/85
calculating (35, 0.05) - run 26/85
calculating (35, 0.1) - run 27/85
calculating (35, 0.3) - run 28/85
calculating (35, 0.5) - run 29/85
calculating (35, 0.7) - run 30/85
calculating (40, 0.05) - run 31/85
calculating (40, 0.1) - run 32/85
calculating (4

calculating (100, 0.7) - run 70/85
calculating (150, 0.05) - run 71/85
calculating (150, 0.1) - run 72/85
calculating (150, 0.3) - run 73/85
calculating (150, 0.5) - run 74/85
calculating (150, 0.7) - run 75/85
calculating (200, 0.05) - run 76/85
calculating (200, 0.1) - run 77/85
calculating (200, 0.3) - run 78/85
calculating (200, 0.5) - run 79/85
calculating (200, 0.7) - run 80/85
calculating (250, 0.05) - run 81/85
calculating (250, 0.1) - run 82/85
calculating (250, 0.3) - run 83/85
calculating (250, 0.5) - run 84/85
calculating (250, 0.7) - run 85/85
Optimal number of neighbors: 10
Optimal min_dist: 0.1
765
calculating (10, 0.05) - run 1/85
calculating (10, 0.1) - run 2/85
calculating (10, 0.3) - run 3/85
calculating (10, 0.5) - run 4/85
calculating (10, 0.7) - run 5/85
calculating (15, 0.05) - run 6/85
calculating (15, 0.1) - run 7/85
calculating (15, 0.3) - run 8/85
calculating (15, 0.5) - run 9/85
calculating (15, 0.7) - run 10/85
calculating (20, 0.05) - run 11/85
calculating

calculating (60, 0.5) - run 49/85
calculating (60, 0.7) - run 50/85
calculating (70, 0.05) - run 51/85
calculating (70, 0.1) - run 52/85
calculating (70, 0.3) - run 53/85
calculating (70, 0.5) - run 54/85
calculating (70, 0.7) - run 55/85
calculating (80, 0.05) - run 56/85
calculating (80, 0.1) - run 57/85
calculating (80, 0.3) - run 58/85
calculating (80, 0.5) - run 59/85
calculating (80, 0.7) - run 60/85
calculating (90, 0.05) - run 61/85
calculating (90, 0.1) - run 62/85
calculating (90, 0.3) - run 63/85
calculating (90, 0.5) - run 64/85
calculating (90, 0.7) - run 65/85
calculating (100, 0.05) - run 66/85
calculating (100, 0.1) - run 67/85
calculating (100, 0.3) - run 68/85
calculating (100, 0.5) - run 69/85
calculating (100, 0.7) - run 70/85
calculating (150, 0.05) - run 71/85
calculating (150, 0.1) - run 72/85
calculating (150, 0.3) - run 73/85
calculating (150, 0.5) - run 74/85
calculating (150, 0.7) - run 75/85
calculating (200, 0.05) - run 76/85
calculating (200, 0.1) - run 77

calculating (35, 0.3) - run 28/85
calculating (35, 0.5) - run 29/85
calculating (35, 0.7) - run 30/85
calculating (40, 0.05) - run 31/85
calculating (40, 0.1) - run 32/85
calculating (40, 0.3) - run 33/85
calculating (40, 0.5) - run 34/85
calculating (40, 0.7) - run 35/85
calculating (45, 0.05) - run 36/85
calculating (45, 0.1) - run 37/85
calculating (45, 0.3) - run 38/85
calculating (45, 0.5) - run 39/85
calculating (45, 0.7) - run 40/85
calculating (50, 0.05) - run 41/85
calculating (50, 0.1) - run 42/85
calculating (50, 0.3) - run 43/85
calculating (50, 0.5) - run 44/85
calculating (50, 0.7) - run 45/85
calculating (60, 0.05) - run 46/85
calculating (60, 0.1) - run 47/85
calculating (60, 0.3) - run 48/85
calculating (60, 0.5) - run 49/85
calculating (60, 0.7) - run 50/85
calculating (70, 0.05) - run 51/85
calculating (70, 0.1) - run 52/85
calculating (70, 0.3) - run 53/85
calculating (70, 0.5) - run 54/85
calculating (70, 0.7) - run 55/85
calculating (80, 0.05) - run 56/85
calculat

calculating (15, 0.1) - run 7/85
calculating (15, 0.3) - run 8/85
calculating (15, 0.5) - run 9/85
calculating (15, 0.7) - run 10/85
calculating (20, 0.05) - run 11/85


In [ ]:
print(f"Optimal number of neighbors: {n_neighbors_opt}")
print(f"Optimal min_dist: {min_dist_opt}")

sns.lineplot(data=scdeed_result, x="n_neighbors", y="num_dubious", hue="min_dist", palette="tab10")
plt.title("scDEED results")
plt.ylabel("Number of dubiously embedded cells")
plt.show()

In [ ]:
# Bsub_minmed: 674